In [ ]:
import numpy as np
import torch
torch.set_default_dtype(torch.float64)

import h5py as h5

import tdg
import tdg.HMC as HMC

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
storage = 'ensemble.h5'

Let's study a small, hot example for computational simplicity.

In [ ]:
nx = 5
lattice = tdg.Lattice(nx)

ere = tdg.EffectiveRangeExpansion(torch.tensor([1.0]))

tuning = tdg.AnalyticTuning(ere, lattice)
print(f'{tuning.C[0]}')

In [ ]:
nt = 8
beta = torch.tensor(0.04)
mu = torch.tensor(3.125)
h  = torch.tensor([0,0,0], dtype=torch.float64)

S = tuning.Action(nt, beta, mu, h)

We could in principle use one Hamiltonian to do do the HMC Metropolis-Hastings accept/reject step and another to do the molecular dynamics integration.

Here we use the same Hamiltonian for both.

In [ ]:
H = HMC.Hamiltonian(S)
integrator = HMC.Omelyan(H, 20, 1)
hmc = HMC.MarkovChain(H, integrator)

Let's start from a configuration sampled from the "quenched" distribution, which ignores the fermion determinant.

In [ ]:
ensemble = tdg.ensemble.GrandCanonical(S).generate(100, hmc, start='hot', progress=tqdm)

We can visualize an observable.

In [ ]:
import tdg.plot as visualize

In [ ]:
viz = visualize.History()
viz.plot(ensemble.N.real)

Let's write it to storage and read it back in.

In [ ]:
with h5.File(storage, 'w') as f:
    ensemble.to_h5(f.create_group('/example'))
with h5.File(storage, 'r') as f:
    ENSEMBLE = tdg.ensemble.GrandCanonical.from_h5(f['/example'])

And now perhaps we wish to visualize some OTHER observables, like the action itself and the bosonic number estimator.

In [ ]:
viz = visualize.History(2)
viz.plot(ENSEMBLE.S.real, 0)
viz.plot(ENSEMBLE.N_bosonic.real, 1)

How do the bosonic and fermionic number estimators compare?

In [ ]:
comparison = visualize.ScatterTriangle(2)
comparison.plot(
    (ENSEMBLE.N_bosonic.real,
     ENSEMBLE.N.real
    ))